# Orbital Parameters Calculator

Get the parameters for an elliptical orbit.

## Compute Internal Units

Get the units for the simulation.

In [51]:
GRAVITY = 6.6743e-8 # 6.6743 × 10-8 cm3 g-1 s-2 (cgs)
LENGTH_UNIT_CM =  3.085678e21 # 1 kpc / h
MASS_UNIT_G = 1.989e43 # 10^10 M_sun /  h
VELOCITY_UNIT_CM_S = 1.0e5 # 1 km / s
TIME_UNIT_S = LENGTH_UNIT_CM / VELOCITY_UNIT_CM_S
INTERNAL_G = (GRAVITY / LENGTH_UNIT_CM**3) * MASS_UNIT_G * TIME_UNIT_S**2

## Set Orbital Parameters

Specify the orbit.

In [49]:
import numpy as np
radius = 100 # 100 kpc / h
m1 = 10
m2 = 10

x1 = np.array([radius/2, 0., 0.])
x2 = np.array([-radius/2, 0., 0.])
r_hat = x2 - x1

g_acc_1 = r_hat * (INTERNAL_G * m2) / np.linalg.norm(r_hat)**3

# g_acc = v**2 / r
# v_circ_1 = (np.linalg.norm(g_acc_1) * radius)**0.5
v_circ_1 = (INTERNAL_G * m1 / (2*radius))**0.5
print(r_hat, g_acc_1, v_circ_1, v_circ_1 * VELOCITY_UNIT_CM_S / (LENGTH_UNIT_CM / TIME_UNIT_S))

[-100.    0.    0.] [-43.02193132   0.           0.        ] 46.379915546878124 46.37991554687812


In [38]:
period = 2. * np.pi * radius / v_circ_1
period

np.float64(9.579325200943407)

In [57]:
def generate_initial_conditions(M1, M2, a, e):
    """
    Generates initial positions and velocities for a two-body system with given eccentricity.
    
    Parameters
    ----------
    M1 : float
        Mass of the first body in 10^10 M_sun / h.
    M2 : float
        Mass of the second body in 10^10 M_sun / h.
    a : float
        Semi-major axis of the orbit in kpc / h.
    e : float
        Eccentricity of the orbit (0 <= e < 1 for an elliptical orbit).
    
    Returns
    -------
    pos1, pos2 : np.ndarray
        Initial positions of the two bodies in kpc / h.
    vel1, vel2 : np.ndarray
        Initial velocities of the two bodies in km / s.
    """
    # Periapsis distance
    r_p = a * (1 - e)

    # Calculate the reduced masses' positions (center of mass at origin)
    pos1_x = -r_p * M2 / (M1 + M2)
    pos2_x = r_p * M1 / (M1 + M2)

    # Initial positions (in x-axis, periapsis)
    pos1 = np.array([pos1_x, 0.0, 0.0])
    pos2 = np.array([pos2_x, 0.0, 0.0])

    # Periapsis velocities based on vis-viva equation
    v_p = np.sqrt(INTERNAL_G * (M1 + M2) * (1 + e) / (a * (1 - e)))
    
    # Direction of velocities (perpendicular to the radius vector at periapsis)
    vel1_y = v_p * M2 / (M1 + M2)
    vel2_y = -v_p * M1 / (M1 + M2)

    # Initial velocities (in y-axis)
    vel1 = np.array([0.0, vel1_y, 0.0])
    vel2 = np.array([0.0, vel2_y, 0.0])

    return pos1, pos2, vel1, vel2

# Example usage
M1 = 25. 
M2 = 1.
a = 30.
e = 0.95 # Crazy high ellipticity

pos1, pos2, vel1, vel2 = generate_initial_conditions(M1, M2, a, e)
pos1, pos2, vel1, vel2

(array([-0.05769231,  0.        ,  0.        ]),
 array([1.44230769, 0.        , 0.        ]),
 array([ 0.        , 46.37991555,  0.        ]),
 array([    0.        , -1159.49788867,     0.        ]))